## Evaluation of Monthly Performance based on MASE & SMAPE

In [16]:
!pip install utilsforecast -q

In [17]:
import pandas as pd
import numpy as np
from utilsforecast.losses import smape, mase

In [18]:
train = pd.read_csv("VN1_monthly_train.csv")
actuals=pd.read_csv("Test_VN1_Actuals_final.csv")
forecast=pd.read_csv("final_forecast_monthly_merged.csv")

In [19]:
train["ds"]=pd.to_datetime(train["ds"])
actuals["ds"]=pd.to_datetime(actuals["ds"])
forecast["ds"]=pd.to_datetime(forecast["ds"])

In [20]:
def evaluate(train, actuals, forecast, seasonality=12, id_col='unique_id', target_col='y'):
  fcst_and_actuals = pd.merge(actuals, forecast, on=['ds', 'unique_id'], how='left')
  models=[ "AutoARIMA", "AutoETS", "AutoTBATS", 'RandomForestRegressor','XGBRegressor', "LGBMRegressor", "TFT", "NHITS", "TimeGPT",'TimeGPT_finetuned', "TimesFM"]
  mase_val = mase(fcst_and_actuals, models=models, seasonality=seasonality, train_df=train, id_col='unique_id', target_col='y')
  mase_val = mase_val.mean(numeric_only=True)
  smape_val = smape(fcst_and_actuals, models=models,  id_col='unique_id', target_col='y')
  smape_val = smape_val.mean(numeric_only=True)
  results = pd.DataFrame({'mase': mase_val, 'smape': smape_val}, index=models)
  results.index.name = 'models'
  results = results.round(4)
  return results

In [21]:
weekly_results=evaluate(train, actuals, forecast, seasonality=52, id_col='unique_id', target_col='y')

In [23]:
weekly_results

,MASE,SMAPE
AutoARIMA,0.4359,0.1713
AutoETS,0.3765,0.1723
AutoTBATS,0.6998,0.2485
RandomForestRegressor,0.4687,0.1710
XGBRegressor,0.5761,0.1861
LGBMRegressor,0.5849,0.1877
TFT,0.3728,0.1759
NHITS,0.3692,0.1746
TimeGPT,0.3802,0.1649
TimeGPT_finetuned,NaN,NaN
